## Business Data Science Homework 5
### Finished by Zhuo Wen & Hao Lun Chu Colin

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('NYC_stop_and_frisk.dat',sep=' ',skiprows=6)
data

,stops,pop,past.arrests,precinct,eth,crime
0,75,1720,191,1,1,1
1,36,1720,57,1,1,2
2,74,1720,599,1,1,3
3,17,1720,133,1,1,4
4,37,1368,62,1,2,1
...,...,...,...,...,...,...
895,2,3233,4,75,2,4
896,111,61692,181,75,3,1
897,48,61692,38,75,3,2
898,115,61692,61,75,3,3


## 2. What fraction of the total stops correspond to “white/back/hispanic”? What fraction of the population corresponds to “white/black/hispanic”?

In [3]:
totalstop = data['stops'].sum()
totalpopulation = data['pop'].sum()
eth=['','Black','Hispanic','White']
for x in data['eth'].unique():
    ethstop = data.loc[data.eth==x,'stops'].sum()
    ethpopulation = data.loc[data.eth==x,'pop'].sum()
    print('The fraction of the total stops correspond to',eth[x],'is',ethstop/totalstop)
    print('The fraction of the total population correspond to',eth[x],'is',ethpopulation/totalpopulation)
    print()

The fraction of the total stops correspond to Black is 0.5312966063004109
The fraction of the total population correspond to Black is 0.27546911368656585

The fraction of the total stops correspond to Hispanic is 0.3395449703241516
The fraction of the total population correspond to Hispanic is 0.2557414035070107

The fraction of the total stops correspond to White is 0.12915842337543754
The fraction of the total population correspond to White is 0.46878948280642346



## 3. Use a Poisson regression to model the number of stops, controlling for ethnicity and using the number of past arrests as an exposure input.

In [4]:
import patsy as pt
import pymc3 as pm
import pandas as pd
from patsy import dmatrices
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [5]:
df = pd.get_dummies(data, columns=['eth'])
df = df.rename(columns={'eth_1': "black", 'eth_2': "hispanic", 'eth_3': "white",'past.arrests':"arrests"})
df = df[df['arrests']>0]
df.head()

,stops,pop,arrests,precinct,crime,black,hispanic,white
0,75,1720,191,1,1,1,0,0
1,36,1720,57,1,2,1,0,0
2,74,1720,599,1,3,1,0,0
3,17,1720,133,1,4,1,0,0
4,37,1368,62,1,1,0,1,0


In [6]:
expr = """stops ~ black + hispanic + pop"""
y, X = dmatrices(expr, df, return_type='dataframe')
poisson_training_results = sm.GLM(y, X, family=sm.families.Poisson(), exposure = df['arrests']).fit()
print(poisson_training_results.summary())

Generalized Linear Model Regression Results                  
Dep. Variable:                  stops   No. Observations:                  899
Model:                            GLM   Df Residuals:                      895
Model Family:                 Poisson   Df Model:                            3
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -94255.
Date:                Thu, 17 Oct 2019   Deviance:                   1.8317e+05
Time:                        23:52:51   Pearson chi2:                 2.79e+05
No. Iterations:                     6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.8004      0.009    -89.294      0.000      -0.818   

### 4. According to the output of your model, what fraction fewer or more stops does each ethnicity have with respect to the others, in proportion to arrest rates of the previous year? Note that you can just pick a baseline ethnicity and just compare everything to that.

In [7]:
black = np.exp(poisson_training_results._results.params['x1'])
hispanic = np.exp(poisson_training_results._results.params['x2'])
print('black = '+str(black))
print('hispanic = '+str(hispanic))
print('hispanic & white  diff = '+ str(hispanic))
print('black & white diff = '+ str(black))

black = 1.1889612578195912
hispanic = 1.279474797670672
hispanic & white  diff = 1.279474797670672
black & white diff = 1.1889612578195912


### Compare to white, hispanic are 1.2794 more likely to get stopped and black are 1.188 more likely to get stopped.

### 5. Next, add the 75 precincts, and again solve the Poisson regression model.

In [8]:
df = pd.get_dummies(df, columns=['precinct'])
df = df[df['arrests']>1]
df.head()

,stops,pop,arrests,crime,black,hispanic,white,precinct_1,precinct_2,precinct_3,...,precinct_66,precinct_67,precinct_68,precinct_69,precinct_70,precinct_71,precinct_72,precinct_73,precinct_74,precinct_75
0,75,1720,191,1,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,36,1720,57,2,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,74,1720,599,3,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,17,1720,133,4,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,37,1368,62,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
expr = """stops ~ pop + black + hispanic + precinct_1 + precinct_2 + precinct_3 + precinct_4 + precinct_5 + precinct_6 + precinct_7 + precinct_8 + precinct_9 + precinct_10 + precinct_11 + precinct_12 + precinct_13 + precinct_14 + precinct_15 + precinct_16 + precinct_17 + precinct_18 + precinct_19 + precinct_20 + precinct_21 + precinct_22 + precinct_23 + precinct_24 + precinct_25 + precinct_26 + precinct_27 + precinct_28 + precinct_29 + precinct_30 + precinct_31 + precinct_32 + precinct_33 + precinct_34 + precinct_35 + precinct_36 + precinct_37 + precinct_38 + precinct_39 + precinct_40 + precinct_41 + precinct_42 + precinct_43 + precinct_44 + precinct_45 + precinct_46 + precinct_47 + precinct_48 + precinct_49 + precinct_50 + precinct_51 + precinct_52 + precinct_53 + precinct_54 + precinct_55 + precinct_56 + precinct_57 + precinct_58 + precinct_59 + precinct_60 + precinct_61 + precinct_62 + precinct_63 + precinct_64 + precinct_65 + precinct_66 + precinct_67 + precinct_68 + precinct_69 + precinct_70 + precinct_71 + precinct_72 + precinct_73 + precinct_74 + precinct_75"""
y, X = dmatrices(expr, df, return_type='dataframe')
poisson_training_results = sm.GLM(y, X, family=sm.families.Poisson(), exposure = df['arrests']).fit()
print(poisson_training_results.summary())

Generalized Linear Model Regression Results                  
Dep. Variable:                  stops   No. Observations:                  898
Model:                            GLM   Df Residuals:                      820
Model Family:                 Poisson   Df Model:                           77
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -72931.
Date:                Thu, 17 Oct 2019   Deviance:                   1.4053e+05
Time:                        23:53:12   Pearson chi2:                 2.15e+05
No. Iterations:                     7                                         
Covariance Type:            nonrobust                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -1.1278      0.013    -83.904      0.000      -1.154

In [10]:
black = np.exp(poisson_training_results._results.params['x2'])
hispanic = np.exp(poisson_training_results._results.params['x3'])
print('black = '+str(black))
print('hispanic = '+str(hispanic))
print('hispanic & white diff = '+ str(hispanic))
print('black & white diff = '+ str(black))

black = 1.7834025417699364
hispanic = 1.7607759094171689
hispanic & white diff = 1.7607759094171689
black & white diff = 1.7834025417699364


### After considering the precincts, by having whote as the baseline, hispanic are 1.7607 likely to get stopped, whereas black are 1.7834 less likely to get stopped.

## Problem 2

In [11]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
import math

In [12]:
nba = pd.read_csv('nba_cc_fake_data.csv', index_col=0)
nba

,Comp,Height,Points,Salary
0,9.0,76.0,27.0,0.0
1,7.0,78.0,39.0,0.0
2,9.0,76.0,39.0,0.0
3,9.0,74.0,39.0,0.0
4,9.0,74.0,26.0,0.0
...,...,...,...,...
9995,8.0,76.0,43.0,0.0
9996,8.0,78.0,25.0,0.0
9997,9.0,77.0,31.0,0.0
9998,8.0,74.0,37.0,0.0


### 1. Explain why linear regression is not appropriate, given the nature of the data.

Linear regression is not appropriate here since there is no linear relationship between three features we are given and the salary. No matter how good a player is in high school, his salary will be zero if he didn't go to NBA. Besides that,all three features we have are not continuous value but are discrete or categorical variablem, making linear regression less accurate and meaningful.

### 2. Try least squares regression, anyway.  How well do you do?

In [13]:
X = nba[['Comp','Height','Points']]
y = nba['Salary']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [14]:
ols = LinearRegression().fit(X_train, y_train)
ols.score(X_test, y_test)

0.18297542118405608

As we can see from the model ouput, the performance is really really bad, given the accuracy score is less than 20%. We have enough confidence to say the linear model is not approproiate predicting player's salary in this case.

### 3-1 Build a model that predicts the probability of making it to the NBA.

In [15]:
nba['Make_nba'] = np.where(nba['Salary']>0,1,0)
X1 = nba[['Comp','Height','Points']]
y1 = nba['Make_nba']

### 3-2 Do a train-test split of 8000/2000 points, train your best model on the training set, and compute the AUC on the test set.

In [16]:
## We tried logistic regression
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.2, random_state=42)
lr = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(X1_train, y1_train)
lr.score(X1_test,y1_test)

0.957

In [17]:
## We tried random forest
rf = RandomForestClassifier(n_estimators=4, max_depth=5).fit(X1_train, y1_train)
rf.score(X1_test,y1_test)

0.952

In [18]:
## We tried gradient boosting
gb = GradientBoostingClassifier(n_estimators=30, max_depth=5).fit(X1_train, y1_train)
gb.score(X1_test,y1_test)

0.956

### We found logistic regression has the best performance, therefore, we choose logistic regression as our model.

In [19]:
roc=roc_auc_score(y1_test, lr.predict_proba(X1_test)[:,1])
print('The AUC on test set is '+str(roc))

The AUC on test set is 0.9554735022585427


### 3-3 Now, build a model to predict the salary.  Note that you may wish to consider a non-linear transformation of your data. What is your R2 score on the test set?

In [20]:
nbaplayer = nba[nba.Make_nba ==1]
nbaplayer

,Comp,Height,Points,Salary,Make_nba
9,9.0,78.0,63.0,496315.0,1
27,10.0,76.0,52.0,741265.0,1
30,8.0,78.0,62.0,292281.0,1
115,10.0,77.0,63.0,782210.0,1
117,8.0,76.0,50.0,259365.0,1
...,...,...,...,...,...
9918,9.0,76.0,43.0,301898.0,1
9928,10.0,79.0,47.0,949052.0,1
9962,10.0,79.0,52.0,557551.0,1
9967,10.0,78.0,54.0,751278.0,1


In [21]:
## First we tried Standardize the data
X_scaler = StandardScaler()
X2 = nbaplayer[['Comp','Height','Points']]
y2 = nbaplayer['Salary']
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=42)

X2_train_std = X_scaler.fit_transform(X2_train)
X2_test_std = X_scaler.transform(X2_test)

ols = LinearRegression().fit(X2_train_std, y2_train)
y_pred = ols.predict(X2_test_std)
r2_score(y2_test, y_pred)

0.5415468455217294

### It seems like standardize the data doesn't really help us, so we tried log transformation.

In [22]:
X2 = np.log(X2)
y2 = np.log(y2)
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=42)

In [23]:
## We tried logistic regression
ols = LinearRegression().fit(X2_train, y2_train)
y_pred = ols.predict(X2_test)
print('The r square score of logistic regression is '+str(r2_score(y2_test, y_pred)))

The r square score of logistic regression is 0.6307385310504816


In [24]:
## Again, we tried random forest
from sklearn.ensemble import RandomForestRegressor
rf1 = RandomForestRegressor(n_estimators=5, max_depth=5).fit(X2_train, y2_train)
y_pred = rf1.predict(X2_test)
print('The r square score of random forest is '+str(r2_score(y2_test, y_pred)))

The r square score of random forest is 0.6040049393942528


### Again, we choose logistic regression since it has better performance.

### 4. Compute  the  expected  NBA  salary  of  a  high  school  basketball  player  who  is  6’  6”  tall,  is averaging 46 points per game, and is playing in the second most competitive league (comp =9), according to your model.

In [25]:
## a player will make into NBA if the probability of making it to the NBA is higher than 
## the probablity parameter in this function
def playersal(height, points, league):
    sal = ols.predict([[np.log(height),np.log(points), np.log(league)]])
    expected_sal = math.exp(sal)*lr.predict_proba([[height,points, league]])[0][1]
    print('The probablity of this play making NBA is '+str(lr.predict_proba([[height,points, league]])[0][1]))
    print('The expected salary of this player in the NBA is '+ str(expected_sal))

In [26]:
playersal(9,78,46)

The probablity of this play making NBA is 0.10033141526796388
The expected salary of this player in the NBA is 46719.017820924346
